In [2]:
import numpy as np
import qutip as qt
import cmath

In [3]:
# Swap function
def swapPositions(list, pos1, pos2):
     
    list[pos1], list[pos2] = list[pos2], list[pos1]
    return list
 
# Driver function
#List = [23, 65, 19, 90]
pos1, pos2  = 2, 4
List = [0,1,2,3,4]
#print(swapPositions(List, pos1-1, pos2-1))

### BS Function

In [4]:
def BS(J, N, state, number_modes, mode1, mode2):
    #N = 2  # number of levels in qubit state
    #E = 1   # number of photons in the mode
    alpha = 1 # eigenvalue of coherent state
    t = 10  # evolution time
    steps = 1000 #time steps
    #J = 0.07853957008534297  # strength of the COUPLİNG for 50/50
    #kappa = 0  # strength of the Cross-Kerr interaction
    
    
    a = qt.qeye(N)
    b = qt.tensor(qt.destroy(N), qt.qeye(N))
    c = qt.tensor(qt.qeye(N), qt.destroy(N))
    
    H_TDC = J * (b.dag()*c + c.dag()*b)   # TDC, on one mode a new photon is created, on the other one is destroyed.
    tlist = np.linspace(0, t, steps)
    BS_all = qt.propagator(H_TDC, tlist, [], {}) 
    BS = BS_all[-1]
    
    #Creating a list [0,1,2,3,4] for the modes
    initial_permute_list = list(range(number_modes))  #[0,1,2,...]
    
    #Swapping the modes
    
    pos1, pos2  = 1,mode1+1
    
    #swapPositions is already defined above. 
    permuted_list = swapPositions(initial_permute_list, pos1-1, pos2-1)

    pos3, pos4 = 2,mode2+1
    permuted_list2 = swapPositions(permuted_list, pos3-1, pos4-1)

    
    state_perm = state.permute(permuted_list2)
    
    operator_list = [BS] + [a] * (number_modes - 2)

    q1 = qt.tensor(operator_list) * state_perm
    
    q2 = q1.permute(permuted_list)
    
    
    results = []
    results.append(q2)
    for i in range(number_modes):
        pt = q2.ptrace(i)
        
        results.append(np.trace(pt * qt.create(N) * qt.destroy(N)))
        
        
    return results

### PS

In [5]:
def PS(N, phi, state, number_modes, mode1):
    #N = 10  # number of levels in qubit state
    t = 10  # evolution time
    steps = 1000 #time steps
    
    a = qt.qeye(N)
    
    
    #phi = 0.1570909090909091
    x = [np.exp(1j*n*phi) for n in range(N)]
    PS = qt.Qobj(np.diag(x))
    
    #Creating a list [0,1,2,3,4] for the modes
    initial_permute_list = list(range(number_modes))  #[0,1,2,...]
    
    #Swapping the modes
    
    pos1, pos2  = 1,mode1+1
    
    #swapPositions is already defined above. 
    permuted_list = swapPositions(initial_permute_list, pos1-1, pos2-1)
    
    state_perm = state.permute(permuted_list)
    
    operator_list = [PS] + [a] * (number_modes - 1)

    q1 = qt.tensor(operator_list) * state_perm
    
    q2 = q1.permute(permuted_list)
    
    
    results = []
    results.append(q2)
    for i in range(number_modes):
        pt = q2.ptrace(i)
        results.append(np.trace(pt * qt.create(N) * qt.destroy(N)))
        
        
    return results

### CKH

In [6]:
def CKH2(N, kappa, state, number_modes, mode1, mode2,*args): #args := the modes we want to put into the partial trace
    #N = 10
    #alpha = 1
    t = 10  # evolution time
    steps = 1000 #time steps
    #J = 0.07853957008534297  # strength of the COUPLİNG for 50/50
    #kappa = 0  # strength of the Cross-Kerr interaction

    a = qt.qeye(N)
    a_V = qt.tensor(qt.destroy(N), qt.qeye(N))
    c = qt.tensor(qt.qeye(N), qt.destroy(N))
    
    H_int = kappa * a_V.dag() * a_V  * c.dag() * c
    
    tlist = np.linspace(0, t, steps)
    U_all = qt.propagator(H_int, tlist, [], {}) 
    U = U_all[-1]

    #Creating a list [0,1,2,3,4] for the modes
    initial_permute_list = list(range(number_modes))
    
    #Swapping the modes
    
    pos1, pos2  = 1, mode1+1
    
    #swapPositions is already defined above. 
    permuted_list = swapPositions(initial_permute_list, pos1-1, pos2-1)

    pos3, pos4 = 2,mode2+1
    permuted_list2 = swapPositions(permuted_list, pos3-1, pos4-1)
    state_perm = state.permute(permuted_list2)
    
    operator_list = [U] + [a] * (number_modes - 2)

    q1 = qt.tensor(operator_list) * state_perm
    
    q2 = q1.permute(permuted_list)
    
    
    results0 = []
    results0.append(q2)
    
    pt_mode_list = [] 
    for i in args:
        pt_mode_list.append(i)
        
    pt = q2.ptrace(pt_mode_list)
    results0.append(pt)
    alpha = (pt[1][0][N])   #|0E> 

    phi = np.angle(alpha)
     

    results0.append(phi)    
    
        
        
    return results0

## Identity

In [7]:
def identity(N, state, number_modes, mode1):
    
    a = qt.qeye(N)
    

    #Creating a list [0,1,2,3,4] for the modes
    initial_permute_list = list(range(number_modes))  #[0,1,2,...]
    
    #Swapping the modes
    
    pos1, pos2  = 1,mode1+1
    
    #swapPositions is already defined above. 
    permuted_list = swapPositions(initial_permute_list, pos1-1, pos2-1)
    
    state_perm = state.permute(permuted_list)
    
    operator_list = [a] + [a] * (number_modes - 1)

    q1 = qt.tensor(operator_list) * state_perm
    
    q2 = q1.permute(permuted_list)
    
    
    results = []
    results.append(q2)
    for i in range(number_modes):
        pt = q2.ptrace(i)
        results.append(np.trace(pt * qt.create(N) * qt.destroy(N)))
        
        
    return results

### Measurement Function

In [8]:
def meas(N,state,mode1,mode2):
    results = []
    #a = qt.qeye(N)
    pt = state.ptrace(mode1)
    pt2 = state.ptrace(mode2)
    qwe = np.trace(pt * qt.create(N) * qt.destroy(N)) + np.trace(pt2 * qt.create(N) * qt.destroy(N))
    results.append(qwe)
    
    return results



# The Sigma Function Here Are Specifally Defined For The Setup!!

### Defining a Sigma_Z function

In [9]:
def sigmaz_meas(prob_mode1,prob_mode2,initial_state,nm_total,ckh_mode1,ckh_mode2,prep_mode1, prep_mode2,select):

    #state3 = BS(0.07853957008534297,N, state:=state2 ,number_modes,prob_mode1,prob_mode2)[0]
    state3 = CKH2(N, kappa:=1*0.3139464882943144, state:=initial_state, number_modes,ckh_mode1,ckh_mode2,prob_mode1,prob_mode2)[0]

    

    if select == 0:
        return state3
        
    else:
        return CKH2(N, kappa:=1*0.3139464882943144, state:=initial_state, number_modes,ckh_mode1,ckh_mode2,prob_mode1,prob_mode2)[1]

### Defining Sigma_X Function

In [10]:
def sigmax_meas(prob_mode1,prob_mode2,initial_state,nm_total,ckh_mode1,ckh_mode2,prep_mode1, prep_mode2,select):

    #Prep modes are the ones who are used to achieve a certain state (e.g. H) from the initial state (e.g. x+)
    state1 = PS(N, phi:=3*np.pi/2,qt.tensor(initial_state),nm_total,prep_mode1)[0]
    state2 = BS(0.07853957008534297,N, state1 ,number_modes:=nm_total,prep_mode1,prep_mode2)[0]

    #state3 = BS(0.07853957008534297,N, state:=state2 ,number_modes,prob_mode1,prob_mode2)[0]
    state3 = CKH2(N, kappa:=1*0.3139464882943144, state:=state2, number_modes,ckh_mode1,ckh_mode2,prob_mode1,prob_mode2)[0]

    state4 = PS(N, phi:=np.pi,state3, number_modes,prep_mode1)[0]
    #state4 = state2

    #Getting the state back
    state5 = BS(0.07853957008534297,N, state4 ,number_modes:=nm_total,prep_mode1,prep_mode2)[0]
    state6 = PS(N, phi:=3*np.pi/2,state5,nm_total,prep_mode1)[0]
    
    

    if select == 0:
        return state6
        
    else:
        return PS(N, phi:=3*np.pi/2,state5,nm_total,prep_mode1)[1:number_modes+1]
    
    

## Sigma Y 

In [11]:
def sigmay_meas(prob_mode1,prob_mode2,initial_state,nm_total,ckh_mode1,ckh_mode2,prep_mode1, prep_mode2,select):

    #Prep modes are the ones who are used to achieve a certain state (e.g. H) from the initial state (e.g. x+)
    state1 = PS(N, phi:=0*np.pi/2,state:=initial_state,nm_total,prep_mode1)[0]
    state2 = BS(0.07853957008534297,N, state1 ,number_modes:=nm_total,prep_mode1,prep_mode2)[0]

    #state3 = BS(0.07853957008534297,N, state:=state2 ,number_modes,prob_mode1,prob_mode2)[0]
    state3 = CKH2(N, kappa:=1*0.3139464882943144, state:=state2, number_modes,ckh_mode1,ckh_mode2,prob_mode1,prob_mode2)[0]

    state4 = PS(N, phi:=np.pi,state3, number_modes,prob_mode1)[0]

    #Getting the state back
    state5 = BS(0.07853957008534297,N, state4 ,number_modes:=nm_total,prep_mode1,prep_mode2)[0]
    state6 = PS(N, phi,state5,nm_total,prep_mode1)[0]
    
    if select == 0:
        return state6
        
    else:
        return PS(N, phi,state5,nm_total,prep_mode1)[1:number_modes+1]    
    
    

## 1,0,0,0

In [80]:
#initial_state = qt.tensor(qt.basis(N:=2,1),qt.basis(N,0),qt.basis(N,0),qt.basis(N,0)) 

initial_state = 1/np.sqrt(2)*(qt.tensor(qt.basis(N:=2,1),qt.basis(N,0),qt.basis(N,0),qt.basis(N,0)) + qt.tensor(qt.basis(N,0),qt.basis(N,1),qt.basis(N,0),qt.basis(N,0)))
 
#initial_state_1 = qt.tensor(qt.basis(N:=10,0),qt.basis(N,2))
#initial_state_2 = qt.tensor(qt.basis(N:=10,0),qt.basis(N,0))
prob_state_A = qt.tensor(qt.basis(N,1),qt.basis(N,0))
prob_state_B = qt.tensor(qt.basis(N,1),qt.basis(N,0))
#prob_state_A = qt.tensor(qt.babasis(N,1),qt.basis(N,0))
prob_state = qt.tensor(prob_state_A, prob_state_B)

In [74]:
state1 = BS(0.07853957008534297,N, initial_state,4,0,2)[0]
BS(0.07853957008534297,N, initial_state,4,0,2)[1:5]

[(0.4999999999999999+0j),
 (0.4999999999999999+0j),
 (0.4999999999999999+0j),
 (0.4999999999999999+0j)]

In [75]:
state2 = BS(0.07853957008534297,N, state1 ,number_modes:=4,3,1)[0]
BS(0.07853957008534297,N, state1,4,3,1)[1:5]

[(0.4999999999999999+0j),
 (0.4999999999999999+0j),
 (0.4999999999999999+0j),
 (0.4999999999999999+0j)]

### A1

In [76]:
#z_state = sigmaz_meas(prob_mode1:=4,probe_mode2:=5,input_state:=qt.tensor(state2,prob_state),nm_total:=8,ckh_mode1:=4,ckh_mode2:=1,0)
#z_state

In [77]:
ckh_state_A1_1 = BS(0.07853957008534297,N, qt.tensor(state2,prob_state),number_modes:=8,4,5)[0]
BS(0.07853957008534297,N,qt.tensor(state2,prob_state),number_modes,4,5)[1:number_modes+1]

[(0.49999999999999983+0j),
 (0.49999999999999983+0j),
 (0.49999999999999983+0j),
 (0.49999999999999983+0j),
 (0.5000028290693578+0j),
 (0.49999717093064183+0j),
 (0.9999999999999997+0j),
 0j]

In [78]:
ckh_state_A1_2 = BS(0.07853957008534297,N, state:=ckh_state_A1_1,number_modes:=8,6,7)[0]
BS(0.07853957008534297,N,state,number_modes,6,7)[1:number_modes+1]

[(0.4999999999999997+0j),
 (0.4999999999999997+0j),
 (0.4999999999999997+0j),
 (0.4999999999999997+0j),
 (0.5000028290693578+0j),
 (0.49999717093064167+0j),
 (0.5000028290693577+0j),
 (0.4999971709306417+0j)]

In [79]:
path1_A = sigmax_meas(prob_mode1:=4,probe_mode2:=5,initial_state:=ckh_state_A1_2,nm_total:=8,ckh_mode1:=4,ckh_mode2:=1,prep_mode1:=0,prep_mode2:=1,0)

In [61]:
path1_AB = sigmaz_meas(prob_mode1:=6,prob_mode2:=7,path1_A,nm_total,ckh_mode1:=6,ckh_mode2:=1,prep_mode1:=0, prep_mode2:=1,0)

In [62]:
path1_AB_2_B = sigmaz_meas(prob_mode1:=6,prob_mode2:=7,path1_AB,nm_total,ckh_mode1:=3,ckh_mode2:=6,prep_mode1:=2, prep_mode2:=3,0)

In [63]:
path1_AB_2_BA = sigmax_meas(prob_mode1:=4,probe_mode2:=5,initial_state:=path1_AB_2_B,nm_total:=8,ckh_mode1:=3,ckh_mode2:=4,prep_mode1:=2,prep_mode2:=3,0)

In [64]:
ckh_state_A2_4 = BS(0.07853957008534297,N, path1_AB_2_BA ,number_modes,0,2)[0]
BS(0.07853957008534297,N, path1_AB_2_BA ,number_modes,0,2)[1:9]

[(0.12500113381666247+0j),
 (0.2499999999855808+0j),
 (0.37500056146071026+0j),
 (0.24999830473704487+0j),
 (0.500002829069357+0j),
 (0.4999971709306413+0j),
 (0.500002829069357+0j),
 (0.4999971709306413+0j)]

In [65]:
ckh_state_A2_5 = BS(0.07853957008534297,N, ckh_state_A2_4 ,number_modes,3,1)[0]
BS(0.07853957008534297,N, ckh_state_A2_4 ,number_modes,3,1)[1:9]

[(0.12500113381666247+0j),
 (0.12500113380383185+0j),
 (0.37500056146071026+0j),
 (0.3749971709187938+0j),
 (0.5000028290693571+0j),
 (0.4999971709306413+0j),
 (0.500002829069357+0j),
 (0.49999717093064133+0j)]

In [66]:
ckh_state_A2_6 = BS(0.07853957008534297,N, ckh_state_A2_5 ,number_modes,4,5)[0]
BS(0.07853957008534297,N, ckh_state_A2_5 ,number_modes,4,5)[1:9]

[(0.12500113381666245+0j),
 (0.1250011338038318+0j),
 (0.37500056146071015+0j),
 (0.3749971709187937+0j),
 (0.24999943855348833+0j),
 (0.7500005614465098+0j),
 (0.5000028290693569+0j),
 (0.4999971709306411+0j)]

In [67]:
ckh_state_A2_7 = BS(0.07853957008534297,N, ckh_state_A2_6 ,number_modes,6,7)[0]
BS(0.07853957008534297,N, ckh_state_A2_6 ,number_modes,6,7)[1:9]

[(0.12500113381666242+0j),
 (0.12500113380383177+0j),
 (0.37500056146071015+0j),
 (0.37499717091879353+0j),
 (0.24999943855348833+0j),
 (0.7500005614465095+0j),
 (0.4999980240016614+0j),
 (0.5000019759983364+0j)]

In [68]:
n4n6 = qt.tensor(qt.qeye(N),qt.qeye(N),qt.qeye(N),qt.qeye(N),qt.destroy(N)*qt.create(N),qt.qeye(N),qt.destroy(N)*qt.create(N),qt.qeye(N))
n4n7 = qt.tensor(qt.qeye(N),qt.qeye(N),qt.qeye(N),qt.qeye(N),qt.destroy(N)*qt.create(N),qt.qeye(N),qt.qeye(N),qt.destroy(N)*qt.create(N))
n5n6 = qt.tensor(qt.qeye(N),qt.qeye(N),qt.qeye(N),qt.qeye(N),qt.qeye(N),qt.destroy(N)*qt.create(N),qt.destroy(N)*qt.create(N),qt.qeye(N))
n5n7 = qt.tensor(qt.qeye(N),qt.qeye(N),qt.qeye(N),qt.qeye(N),qt.qeye(N),qt.destroy(N)*qt.create(N),qt.qeye(N),qt.destroy(N)*qt.create(N))

In [69]:
ckh_state_A2_7.dag() * n4n6 * ckh_state_A2_7

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.37500367]]

In [70]:
ckh_state_A2_7.dag() * n4n7 * ckh_state_A2_7

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.37499689]]

In [71]:
ckh_state_A2_7.dag() * n5n6 * ckh_state_A2_7

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.1249983]]

In [72]:
ckh_state_A2_7.dag() * n5n7 * ckh_state_A2_7

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.12500113]]